In [5]:
from IPython import display as disp
import torch
import torchaudio
import os
from denoiser import pretrained
from denoiser.dsp import convert_audio
from model import Preprocessor, get_decoder_ngram_model, CMCWav2vec, infer
import glob
from decouple import config
from normalize_text import find_all_text_number, word_2_number
import os
import json

In [6]:
def load_all_model(lm_path,last_checkpoint):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    preprocessor = Preprocessor(last_checkpoint,model_type='large', device=device)
    lm_model = get_decoder_ngram_model(preprocessor.processor.tokenizer, ngram_lm_path=lm_path)
    pytorch_model = CMCWav2vec(last_checkpoint, 'large', device=device)
    return lm_model,pytorch_model,device

In [7]:
model = pretrained.dns64().cuda()
wav, sr = torchaudio.load('/home/doanh/kaggle/naver/submiss/voice/private_test_16k_1ac_vad/tfjjxjn6g6axnedapwqycl5a.wav')
wav = convert_audio(wav.cuda(), sr, model.sample_rate, model.chin)
with torch.no_grad():
    denoised = model(wav[None])[0]
disp.display(disp.Audio(wav.data.cpu().numpy(), rate=model.sample_rate))
disp.display(disp.Audio(denoised.data.cpu().numpy(), rate=model.sample_rate))

In [13]:



lm_model,pytorch_model,device = load_all_model("../ngram_lm/vi_4gram.binary","../wav2vec_model/wav2vec2-large-nguyenvulebinh/checkpoint-80000")
audio_path = "/home/doanh/kaggle/naver/submiss/voice/private_test_16k_1ac_vad/tfjjxjn6g6axnedapwqycl5a.wav"
raw = infer(audio_path,lm_model,pytorch_model)
print(f"raw: {raw}")

wav, sr = torchaudio.load(audio_path)
wav = convert_audio(wav.cuda(), sr, model.sample_rate, model.chin)
with torch.no_grad():
    denoised = model(wav[None])[0]
    torchaudio.save("denoised.wav",denoised.cpu(), sr, encoding="PCM_S", bits_per_sample=16)

norm = infer("denoised.wav",lm_model,pytorch_model)
print(f"norm: {norm}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unigrams not provided and cannot be automatically determined from LM file (only arpa format). Decoding accuracy might be reduced.
No known unigrams provided, decoding results might be a lot worse.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


raw: bật hai hai hai ba ba hắt mười đi ba mươi bảy phần trăm
norm: bật hai hai hai ba ba hắt mười đi ba mươi bảy phần trăm
